## Тестовое задание Авито

Процесс обучения происходил с помощью вычислительных возможностей kaggle (GPU P 100)

In [1]:
!pip install evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.3 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=79a0cbd4e66a8e327472efd79483676fee4f48d4b2ca7052d19ce789d1f624c9
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
ces

### Импортируем зависимости и фиксируем сиды

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset, Features, Value, ClassLabel, Sequence
import pandas as pd
import numpy as np
import os
import random
import torch

Для задачи восстановления пробелов была использована относительно недавняя и современная модель от Google - CANINE. 
Основным преимуществом в пользу выбора ее стал ее маленький размер, способность работать с русским текстом "из коробки" и главное - токенизатор на уровне байтов с ТЕСНОЙ связью с символами (некоторые символы токенизируются одним элементом, для других требуется несколько). Основное преимущество этого состоит в том, что легко выставлять метки для задачи token classification и после выставления легко понимать, где необходимо ставить пробел, в отличие от токенизаторов WordPiece, SentencePiece и BPE "русских" моделей ai-forever и deeppavlov, в которых пробел может попасть внутрь токена. Тем не менее эксперименты с моделями предообученными специально для русского языка тоже прошел.

In [ ]:
SEED = 42
CPU_COUNT = os.cpu_count()
# необходимо поменять путь в kaggle для повторения эскперимента
DATA_PATH = "/kaggle/input/dfghjklgfhj/rec_aaa_title_desc.pq"
TEST_DATA_PATH = "/kaggle/input/test-dataset-avito-internship-v1/dataset_1937770_3.txt"

In [ ]:
def set_seed(seed: int):
    """Фиксирует все сиды для воспроизводимости"""
    # Python
    random.seed(seed)
    
    # Numpy
    np.random.seed(seed)
    
    # PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    # Установки для CuDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Для трансформеров
    try:
        import transformers
        transformers.set_seed(seed)
    except:
        pass

# Установите сид перед началом обучения
set_seed(SEED)

2025-09-21 17:16:38.207681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758474998.421455      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758474998.480724      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
def setup_environment_for_reproducibility(seed: int):
    """Настройка окружения для максимальной воспроизводимости"""
    # Установка сидов
    set_seed(seed)
    
    # Переменные окружения
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    
    # Дополнительные настройки PyTorch
    torch.use_deterministic_algorithms(True, warn_only=True)

# Использование
setup_environment_for_reproducibility(SEED)

### Подготовка данных

В качестве данных были взяты описания объявлений с платформы Авито

In [6]:
# Загрузка токенизатора нужной модели
model_name = "google/canine-s"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

In [ ]:
# Читаем данные и создаем датасет
df = pd.read_parquet(DATA_PATH)

features = Features({
    "descriptions": Value("string"),
})
descriptions_dataset = Dataset.from_dict({"descriptions": df['DescriptionRu'].dropna().tolist()}, features=features)

In [ ]:
# Словари для лэйблов классификации
label2id = {"B": 0, "I": 1}
id2label = {0: "B", 1: "I"}

In [ ]:
def tokenized_and_labels(examples):
    """Функция для токенизации описаний и выставлении лэйблов для пробелов"""
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    
    for desc in examples["descriptions"]:
        original_text = desc.replace('\n', ' ').strip()
        words = original_text.split(" ")

        char_labels = ["I" if char_idx else "B" for word_idx, word in enumerate(words) for char_idx, char in enumerate(word)]
        fused_text = original_text.replace(" ", "")

        labels = [-100]
        for current_char_index, char in enumerate(fused_text[:510]):
            char_encoded = tokenizer.encode(char, add_special_tokens=False)
            num_tokens_for_this_char = len(char_encoded)

            # Метка для текущего символа из нашего списка
            char_label = char_labels[current_char_index]

            # Назначаем метку первому токену (байту) этого символа
            labels.append(label2id[char_label])

            # Назначаем -100 всем последующим токенам (байтам) этого символа
            for _ in range(1, num_tokens_for_this_char):
                labels.append(-100)

        labels.extend((512 - len(labels)) * [-100])

        tokenized = tokenizer(fused_text, truncation=True, padding='max_length', max_length=512)

        input_ids_list.append(tokenized['input_ids'])
        attention_mask_list.append(tokenized['attention_mask'])
        labels_list.append(labels)

    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "labels": labels_list,
    }

In [10]:
dataset = descriptions_dataset.map(tokenized_and_labels, batched=True, batch_size=16, remove_columns=["descriptions"], num_proc=CPU_COUNT)

Map (num_proc=4):   0%|          | 0/435355 [00:00<?, ? examples/s]

In [11]:
# Разделяем на train/validation
train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
dataset_dict = {
    'train': train_testvalid['train'],
    'validation': test_valid['train'],
    'test': test_valid['test']
}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

# загружаем модель с hugginface
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

model.safetensors:   0%|          | 0.00/528M [00:00<?, ?B/s]

Some weights of CanineForTokenClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Метрики

In [13]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    """Функция для получения метрик precision recall f1 accuracy во время обучения"""
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Обучение

In [ ]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    run_name="train_bert",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir='./logs',
    report_to="none",
    seed=SEED,
    dataloader_drop_last=True,
    dataloader_pin_memory=False,
)

In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.045200,0.042782,0.883530,0.890156,0.886830,0.984530
2,0.040100,0.039666,0.890646,0.897406,0.894013,0.985557
3,0.038300,0.039673,0.893040,0.899321,0.896169,0.985881


TrainOutput(global_step=65301, training_loss=0.04577939233338712, metrics={'train_runtime': 28159.037, 'train_samples_per_second': 37.105, 'train_steps_per_second': 2.319, 'total_flos': 3.4313467661697024e+17, 'train_loss': 0.04577939233338712, 'epoch': 3.0})

In [ ]:
# сохраняем модель
trainer.save_model("./canine-space-restoration-final")
tokenizer.save_pretrained("./canine-space-restoration-final")

('./bert-space-restoration-final/tokenizer_config.json',
 './bert-space-restoration-final/special_tokens_map.json',
 './bert-space-restoration-final/added_tokens.json')

In [ ]:
from transformers import pipeline

# Загружаем нашу обученную модель
restore_pipe = pipeline("token-classification", model="./canine-space-restoration-final", tokenizer=tokenizer, aggregation_strategy="simple")

def restore_spaces_bert(text):
    # Получаем предсказания модели
    predictions = restore_pipe(text)
    # predictions содержит список {'word': 'к', 'entity': 'B', 'score': 0.999, ...}
    # Собираем результат, вставляя пробелы перед каждым токеном с меткой 'B'
    print(predictions)
    result = []
    for pred in predictions:
        if pred['entity_group'] == 'B' and result: # Если нашли начало нового слова и это не самое первое слово
            result.append(' ')
        result.append(pred['word'].replace('##', ''))
    return ''.join(result)

# Пример использования
test_text = "книгавхорошемсостоянии"
restored_text = restore_spaces_bert(test_text)
print(f"Input: {test_text}")
print(f"Output: {restored_text}")
# Output: "книга в хорошем состоянии"

Device set to use cuda:0


[{'entity_group': 'B', 'score': 0.9999769, 'word': 'к', 'start': None, 'end': None}, {'entity_group': 'I', 'score': 0.99998844, 'word': 'нига', 'start': None, 'end': None}, {'entity_group': 'B', 'score': 0.9999635, 'word': 'вх', 'start': None, 'end': None}, {'entity_group': 'I', 'score': 0.9999998, 'word': 'орошем', 'start': None, 'end': None}, {'entity_group': 'B', 'score': 0.99990857, 'word': 'с', 'start': None, 'end': None}, {'entity_group': 'I', 'score': 0.99999964, 'word': 'остоянии', 'start': None, 'end': None}]
Input: книгавхорошемсостоянии
Output: книга вхорошем состоянии


### Блок для скачивания модели с kaggle

In [ ]:
!zip -r model.zip /kaggle/working/canine-space-restoration-final

  adding: kaggle/working/checkpoint-65301/ (stored 0%)
  adding: kaggle/working/checkpoint-65301/tokenizer_config.json (deflated 73%)
  adding: kaggle/working/checkpoint-65301/model.safetensors (deflated 7%)
  adding: kaggle/working/checkpoint-65301/config.json (deflated 49%)
  adding: kaggle/working/checkpoint-65301/scheduler.pt (deflated 56%)
  adding: kaggle/working/checkpoint-65301/rng_state.pth (deflated 25%)
  adding: kaggle/working/checkpoint-65301/special_tokens_map.json (deflated 82%)
  adding: kaggle/working/checkpoint-65301/trainer_state.json (deflated 76%)
  adding: kaggle/working/checkpoint-65301/optimizer.pt (deflated 25%)
  adding: kaggle/working/checkpoint-65301/training_args.bin (deflated 52%)


In [ ]:
%cd /kaggle/working

In [21]:
from IPython.display import FileLink

In [22]:
FileLink('/kaggle/working/model.zip')

/kaggle/working/model.zip

### Получаем предикт на тестовых данных и сохраняем результат

In [ ]:
def preprocess_file(file_path):
    """Функция для чтения файла с тестовыми данными. (запятые используется и как сепаратор для данных и в самих данных)"""
    cleaned_lines = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split(',')
            if len(parts) > 2:
                cleaned_line = [parts[0], ','.join(parts[1:])]
                cleaned_lines.append(cleaned_line)
            else:
                cleaned_lines.append(parts)
    return cleaned_lines

cleaned_data = preprocess_file(TEST_DATA_PATH)
test_df = pd.DataFrame(cleaned_data[1:])

Результат получился в районе 90 для в F1 метрики на валидационных данных и на данных в тестовом задании. Думаю, что это связано в первую очередь со спецификой оформления объявлений Авито. В тестовых данных можно увидеть короткие фразы/предложения без пробелов. Встречаются фразы не характерные объявлениям, а характерны более художественному/разговорному стилю. Необходимо добавить в датасет "литературных" данных и почистить объявления от специфики, характерной им, чтобы получить метрику на тестовом выше.
К сожалению, на платформе степик я нажал на кнопку получить тестовое только после того, как уже имел модели "на руках" и ограничение на использование GPU на kaggle :), из-за возможного ограничения времени на одну попытку (надпись Time Limit) :)